In [1]:
import pandas as pd
from pandas import DataFrame

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams

from sklearn.preprocessing import FunctionTransformer

from tqdm.auto import tqdm

from scopy.ScoTox import Toxfilter

from itertools import groupby

<frozen importlib._bootstrap>:228: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.


In [2]:
molecules = [mol for mol in Chem.SDMolSupplier("Adenosine_20323.sdf") if mol is not None]
print(f'Количество молекул = {len(molecules)}')

Количество молекул = 20323


In [3]:
Filter = Toxfilter(molecules, detail=True, showSMILES=True)
filter_result = Filter.Check_Toxicophores()
print(filter_result[0:6])

[{'SMILES': 'CCOC(=O)c1c(C)[nH]c(C)c1S(=O)(=O)NCC1CCN(Cc2ccccc2OC)CC1', 'Disposed': 'Rejected', 'MatchedAtoms': [((0, 1, 4, 5, 3),)], 'MatchedNames': ['pyrrole'], 'Endpoint': 'Toxicophores'}, {'SMILES': 'Cc1oc2ncn(C)c(=O)c2c1C(=O)N(Cc1ccccc1)C(C)C', 'Disposed': 'Rejected', 'MatchedAtoms': [((2, 0, 1, 5, 7),)], 'MatchedNames': ['furan'], 'Endpoint': 'Toxicophores'}, {'SMILES': 'COC(=O)c1ccccc1NC(=O)CCCn1nc(C)c2c(C)n(-c3ccccc3)nc2c1=O', 'Disposed': 'Rejected', 'MatchedAtoms': [((3, 15, 26, 31, 33, 32, 27), (12, 10, 9, 22, 29, 30, 24))], 'MatchedNames': ['masked_aniline'], 'Endpoint': 'Toxicophores'}, {'SMILES': 'CCOC(=O)c1cccc(NC(=O)CCCn2nc(C)c3c(C)n(-c4ccccc4)nc3c2=O)c1', 'Disposed': 'Rejected', 'MatchedAtoms': [((3, 12, 28, 33, 34, 32, 29), (14, 16, 15, 10, 25, 26, 27))], 'MatchedNames': ['masked_aniline'], 'Endpoint': 'Toxicophores'}, {'SMILES': 'CCOC(=O)CNC(=O)c1cc2c(=O)oc3ccccc3c2s1', 'Disposed': 'Rejected', 'MatchedAtoms': [((11, 3, 0, 1, 6, 17, 20, 21, 18, 8, 7),), ((2, 1, 0, 5, 4

In [4]:
filter_result_groupby = groupby(sorted(filter_result, key=lambda x: x['Disposed']), lambda x: x['Disposed'])
for k, g in filter_result_groupby:
     print(k, len(list(g)))

Accepted 2361
Rejected 17962


In [5]:
disposed_status = []
for mol in filter_result:
    disposed_status.append(mol.get('Disposed'))

In [6]:
# это ужасный велосипед, который и сейчас кажется мне сущим кошмаром, но он сработал

SMILES = []
index_list = []
i = 0

with open('Adenosine_20323.sdf', 'r', encoding='utf-8') as f:
    for index, line in enumerate(f):
        if "<Smile>" in line:
            index_list.append(index)

SMILES_index = [x+1 for x in index_list]

with open('Adenosine_20323.sdf', 'r', encoding='utf-8') as f:
    for index, line in enumerate(f):
        if index in SMILES_index:
            SMILES.append(line)

SMILES = [s.rstrip() for s in SMILES]
print(len(SMILES))

20323


In [7]:
# создаем словарь из дескриторов структуры
ConstDescriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
                    "NHOHCount": Descriptors.NHOHCount,
                    "NOCount": Descriptors.NOCount,
                    "NumHAcceptors": Descriptors.NumHAcceptors,
                    "NumHDonors": Descriptors.NumHDonors,
                    "NumHeteroatoms": Descriptors.NumHeteroatoms,
                    "NumRotatableBonds": Descriptors.NumRotatableBonds,
                    "NumValenceElectrons": Descriptors.NumValenceElectrons,
                    "NumAromaticRings": Descriptors.NumAromaticRings,
                    "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
                    "RingCount": Descriptors.RingCount}

# создаем словарь из физико-химических дескрипторов                            
PhisChemDescriptors = {"MW": Descriptors.MolWt,
                       "LogP": Descriptors.MolLogP,
                       "MR": Descriptors.MolMR,
                       "TPSA": Descriptors.TPSA}

In [8]:
# объединяем все дескрипторы в один словарь
descriptors = {}
descriptors.update(ConstDescriptors)
descriptors.update(PhisChemDescriptors)
print(f"Количество дескрипторов в словаре: {len(descriptors)}")

Количество дескрипторов в словаре: 15


In [9]:
# функция для генерации дескрипторов из молекул
def mol_dsc_calc(mols): 
    return DataFrame({k: f(m) for k, f in descriptors.items()} 
             for m in mols)

# оформляем sklearn трансформер для использования в конвеерном моделировании (sklearn Pipeline)
descriptors_transformer = FunctionTransformer(mol_dsc_calc, validate=False)

In [10]:
X = descriptors_transformer.transform(molecules)

In [11]:
X['smiles'] = SMILES
X['disposed_status'] = disposed_status
X

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA,smiles,disposed_status
0,32,2,8,6,2,9,9,176,2,1,3,463.600,3.00734,122.4067,100.73,c1(c(c([nH]c1C)C)C(=O)OCC)S(NCC1CCN(Cc2c(OC)cc...,Rejected
1,25,0,6,5,0,6,4,130,3,0,3,339.395,2.88572,95.4385,68.34,c12c(C(N(Cc3ccccc3)C(C)C)=O)c(oc1\N=C/N(C2=O)C)C,Rejected
2,34,1,9,8,1,9,7,174,4,0,4,459.506,3.40454,128.2442,108.11,c12c(c(n(n1)c1ccccc1)C)\C(=N/N(C2=O)CCCC(Nc1c(...,Rejected
3,35,1,9,8,1,9,8,180,4,0,4,473.533,3.79464,132.8612,108.11,c12c(c(n(n1)c1ccccc1)C)\C(=N/N(C2=O)CCCC(Nc1cc...,Rejected
4,23,1,6,6,1,7,4,118,3,0,3,331.349,2.30060,86.8332,85.61,CCOC(CNC(C1SC2=C(C=1)C(Oc1c2cccc1)=O)=O)=O,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20318,22,1,4,4,1,6,4,112,3,0,3,316.357,4.08640,85.1422,51.22,c1cc(c(C(NC2Sc3c(N=2)ccc(c3)OCC)=O)cc1)F,Rejected
20319,24,2,6,6,2,8,3,120,4,0,4,357.416,3.09479,94.1129,79.51,c1cc2N3C(SC(=C3NC(c2cc1)=O)C(NCC1OC=CC=1)=O)=S,Rejected
20320,27,0,6,9,0,9,5,138,4,0,4,417.537,4.34999,110.1840,62.19,c1cc(c(N2c3[nH0]c([nH0](CC4OC=CC=4)c(c3SC2=S)=...,Rejected
20321,29,1,5,6,1,7,7,152,3,0,4,428.579,5.11890,119.2662,60.33,CCOC(C1=C(NC(=O)CSc2c[nH0](CC)c3c2cccc3)SC2=C1...,Rejected


In [12]:
X['disposed_status'].value_counts()

Rejected    17962
Accepted     2361
Name: disposed_status, dtype: int64

In [13]:
filtered_molecules = X.loc[(X.NumHDonors < 5) & (X.MW < 500) & (X.LogP < 5) & 
                           (X.NumHAcceptors < 10) & (X.NumRotatableBonds < 10) & (X.RingCount > 0) &
                           (X.disposed_status == 'Accepted')]

In [14]:
filtered_molecules

,HeavyAtomCount,NHOHCount,NOCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumAliphaticHeterocycles,RingCount,MW,LogP,MR,TPSA,smiles,disposed_status
18,24,1,5,5,1,6,3,124,3,0,4,342.830,4.92880,95.0877,63.84,CC1CCC(Nc2c3c(ON=C3c3ccc(Cl)cc3)[nH0]c[nH0]2)CC1,Accepted
46,24,1,9,8,1,9,6,130,2,0,2,336.348,0.78432,84.6532,116.32,c12c(\N=C(/N(C1=O)CC(NC(CC)C)=O)\C)onc2C(=O)OCC,Accepted
56,30,1,8,7,1,10,6,152,4,0,4,410.384,1.63090,103.3317,94.70,c1c(CNC(C[nH0]2c(c3c([nH0]c2)[nH0]([nH0][nH0]3...,Accepted
84,28,1,7,5,1,7,4,150,2,1,4,383.496,3.20492,106.7257,74.50,n1c(noc1c1ccc(cc1)C)CN1CCN(C(NC2CCCCC2)=O)CC1,Accepted
85,27,1,6,5,1,6,8,144,2,0,3,369.465,4.24550,103.4957,77.25,n1c(noc1c1ccc(OCC(=O)NCC/C/2=C/CCCC2)cc1)C(C)C,Accepted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20254,22,1,6,6,1,6,4,114,3,0,3,296.378,2.34982,89.1367,58.87,Cc1cc(CNc2[nH0]c([nH0]c3c2c[nH0][nH0]3C)N(C)C)...,Accepted
20282,25,1,7,7,1,7,4,130,3,1,4,338.415,2.12042,97.8417,68.10,Cc1ccc(CNc2[nH0]c([nH0]c3c2c[nH0][nH0]3C)N2CCO...,Accepted
20283,25,1,6,5,1,7,6,128,3,0,3,357.797,3.25750,93.2887,77.25,n1c(onc1CNC(=O)Cc1cc(OC)ccc1)c1cc(Cl)ccc1,Accepted
20285,25,2,7,7,2,7,8,132,3,0,3,340.431,2.73222,99.7924,76.89,c1cc(CNc2c3c([nH0]([nH0]c3)C)[nH0]c([nH0]2)NCC...,Accepted


In [16]:
smiles_list = filtered_molecules['smiles'].tolist()
print(len(smiles_list))

2138


In [19]:
with open('accepted_smiles.txt', 'w', encoding='utf-8') as f:
    for item in smiles_list:    
        f.write("%s\n" % item)